In [1]:
# Mac 이나 Linux
import time
import multiprocessing

def count_down(x):
    while True:
        if x == 0:
            break
            
        print("카운트 다운... %d" % x)
        x -= 1
        time.sleep(1)
        
p = multiprocessing.Process(target=count_down, args=(5.))
p.start()

TypeError: 'float' object is not iterable

In [8]:
# Windows 는 요렇게
import time
import multiprocessing

def count_down(x):
    while True:
        if x == 0:
            break
            
        print("카운트 다운 ... %d" % x)
        x -= 1
        time.sleep(1)

if __name__ == '__main__':
    p = multiprocessing.Process(target=count_down, args=(5,))
    print(p)
    p.start()
    p.join()

<Process name='Process-6' parent=10636 initial>


In [10]:
# 프로세스 리스트
# 아래 거 안되면 !ps -ef
# !ps -alx

'ps'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [11]:
# Task가 CPU의 추상화인 이유는?
# 동시에 작동한다 ? --> 정말 동시에 ?
# Multi-Tasking (멀티태스킹) --> 실제로는 시분할로 동작
# 실제로 우리가 구동시키는 모든 프로그램들은 전부 다 하나도 빠짐없이 모두 기계어로 구동된다.
# 이 기계어를 구동시킬 수 있는 녀석은 유일하게 CPU뿐이다.
# 결국 이놈들이 시분할로 동작하면서 경쟁하는 이유는? 
# CPU 자원이 한정되어 있기 때문에

In [19]:
import threading
import os

def info():
    print('module name: ', __name__)
    print('parent process: ', os.getpid())
    print('process id: ', threading.get_ident())
    print('==========Context Switching==========')
    
def print_hello():
    print("안녕!")
    info()
    time.sleep(1)
    print("잠자다 일어나기\n")
    
if __name__ == '__main__':
    start = time.perf_counter()
    threads = []
    
    for _ in range(5):
        t = threading.Thread(target = print_hello)
        t.start()
        threads.append(t)
        
    for thread in threads:
        thread.join()
        
    finish = time.perf_counter()
    
    print('종료: ', str(round(finish - start, 2)) + "초")
    
# window 경우 process 안에 여러개의 쓰레드 존재.
# 동작이 끝나는 것을 보장받고 싶으면 Async await를 사용해야한다.

안녕!
module name:  __main__
parent process:  10636
process id:  14536
==========Context Switching==========
안녕!
module name:  __main__
parent process:  10636
process id:  9716
==========Context Switching==========
안녕!
안녕!module name:  __main__
parent process:  10636
process id:  12760
==========Context Switching==========

module name:  __main__
parent process:  10636
process id:  4272
==========Context Switching==========
안녕!
module name:  __main__
parent process:  10636
process id:  11840
==========Context Switching==========
잠자다 일어나기
잠자다 일어나기
잠자다 일어나기

잠자다 일어나기



잠자다 일어나기

종료:  1.02초


In [22]:
import threading

x = 0

def increment_global():
    global x
    x += 1
    
def thread_processing():
    for _ in range(1000000):
        increment_global()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0}번째 반복 이후 = {1}".format(i, x))
    
# 크리티컬 섹션에서 발생하는 문제를 알아보기 위함
# 크리티컬 섹션 = x
# 실행이 보장받지 않았음

0번째 반복 이후 = 1550445
1번째 반복 이후 = 1873871
2번째 반복 이후 = 1322158
3번째 반복 이후 = 1747640
4번째 반복 이후 = 1734225
5번째 반복 이후 = 1476648
6번째 반복 이후 = 1339039
7번째 반복 이후 = 1464680
8번째 반복 이후 = 1753189
9번째 반복 이후 = 1618428


In [24]:
import threading

lock = threading.Lock()

x = 0

def increment_global():
    global x
    x += 1
    
def thread_processing():
    for _ in range(1000000):
        lock.acquire()
        increment_global()
        lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0}번째 반복 이후 = {1}".format(i, x))

0번째 반복 이후 = 2000000
1번째 반복 이후 = 2000000
2번째 반복 이후 = 2000000
3번째 반복 이후 = 2000000
4번째 반복 이후 = 2000000
5번째 반복 이후 = 2000000
6번째 반복 이후 = 2000000
7번째 반복 이후 = 2000000
8번째 반복 이후 = 2000000
9번째 반복 이후 = 2000000


In [26]:
import threading

lock = threading.Lock()

x = 0

def increment_global():
    global x
    x += 1
    
def thread_processing():
    lock.acquire()
    
    for _ in range(1000000):
        increment_global()
        
    lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0}번째 반복 이후 = {1}".format(i, x))

0번째 반복 이후 = 2000000
1번째 반복 이후 = 2000000
2번째 반복 이후 = 2000000
3번째 반복 이후 = 2000000
4번째 반복 이후 = 2000000
5번째 반복 이후 = 2000000
6번째 반복 이후 = 2000000
7번째 반복 이후 = 2000000
8번째 반복 이후 = 2000000
9번째 반복 이후 = 2000000


In [27]:
# MySQL with Python
!pip install pymysql

In [28]:
import pymysql

In [29]:
MYSQL_USER_DATA_SAVED_FILE = 'mysql/userinfo'

In [32]:
import pickle

mysql_user_info = dict({
    'user_id': 'eddi',
    'password': 'eddi@123'
})

f = open(MYSQL_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(mysql_user_info, f)
f.close()

del mysql_user_info

In [33]:
import pickle

f = open(MYSQL_USER_DATA_SAVED_FILE, 'rb')
mysql_user_info = pickle.load(f)
f.close()

# import pymysql 부터 여기까지는 python 안에 user info를 저장해둔 것.

In [37]:
db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = mysql_user_info['user_id'],
    passwd = mysql_user_info['password'],
    db = 'pydb'
)

print(db)

In [38]:
cursor = db.cursor()
# 커서가 있어야 db를 제어할 수 있다.

sql = """
    create table pydb_test(
        id int unsigned not null auto_increment,
        name varchar(20) not null,
        price int not null,
        primary key(id)
    )
"""

cursor.execute(sql)
# 쿼리 실행
db.commit()
# db에 적용
db.close()
print('테이블 생성 성공!')

테이블 생성 성공!
